In [142]:
using ControlSystems
using DistributedControlSystems
using RobustAndOptimalControl
using LinearAlgebra
using BlockArrays

# Проверка масштабируемости на задаче консенсуса.

Оказалось, что уже при 10 агентах задача не решается.

Это связано с ростом контроллера на каждой итерации, так как предыдущие контроллеры связали первых агентов с текущим:
Они стали наблюдаемыми и управляемыми.

Возможное решение - выделение нулевой динамики на каждой итерации. Если она устойчивая, то можно создать контроллер с размерностью равной относительному порядку.

In [143]:
a = [
    0 1
    0 0
]
b = [
    0
    1
]
c = [1 0]

1×2 Matrix{Int64}:
 1  0

In [144]:
N = 5

5

In [145]:
A = kron(I(N), a)

10×10 Matrix{Int64}:
 0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0

In [146]:
Bs = eachcol(kron(I(N), b)) .|> Array

5-element Vector{Vector{Int64}}:
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [147]:
L = diagm(0 => 2ones(N), 1 => -ones(N - 1), -1 => -ones(N - 1))
L[1, 1] = 1
L[end, end] = 1
L

5×5 Matrix{Float64}:
  1.0  -1.0   0.0   0.0   0.0
 -1.0   2.0  -1.0   0.0   0.0
  0.0  -1.0   2.0  -1.0   0.0
  0.0   0.0  -1.0   2.0  -1.0
  0.0   0.0   0.0  -1.0   1.0

In [148]:
Cs = eachrow(kron(L, c)) .|> Array .|> permutedims

5-element Vector{Matrix{Float64}}:
 [1.0 0.0 … 0.0 0.0]
 [-1.0 -0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … -1.0 -0.0]
 [0.0 0.0 … 1.0 0.0]

In [149]:
ssd = StateSpaceDistributed(A, Bs, Cs)

StateSpaceDistributed{Matrix{Int64}, BlockMatrix{Int64, Matrix{Matrix{Int64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}, BlockMatrix{Float64, Matrix{Matrix{Float64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}, BlockMatrix{Int64, Matrix{Matrix{Int64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}}([0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 1], [1.0 0.0 … 0.0 0.0; -1.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -1.0 -0.0; 0.0 0.0 … 1.0 0.0], [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

In [150]:
fixed_modes(ssd; digits=5)

2-element Vector{ComplexF64}:
 0.0 - 0.0im
 0.0 + 0.0im

In [151]:
cs, Acl = distributed_controllers(ssd)

LoadError: The Hamiltonian matrix is not dichotomic

In [152]:
function null_controller(siso)
    system = minreal2(siso)
    y_system, null_system = null_split(system)
    L = kalman(y_system, I, I)
    K = lqr(y_system, I, I)
    controller = observer_controller(y_system, K, L)
end

null_controller (generic function with 1 method)

In [153]:
ss(0)

StateSpace{Continuous, Int64}
D = 
 0

Continuous-time state-space model

In [154]:
siso = ss(ssd.A, ssd.B[Block(1)], ssd.C[Block(1)], 0)

StateSpace{Continuous, Float64}
A = 
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
B = 
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
C = 
 1.0  0.0  -1.0  -0.0  0.0  0.0  0.0  0.0  0.0  0.0
D = 
 0.0

Continuous-time state-space model

In [155]:
minreal2(siso) |> relative_degree

(2, 1.0)

In [156]:
cs2, Acl2 = distributed_controllers(ssd; alg=null_controller)
Acl2

20×20 Matrix{Float64}:
  0.0      1.0   0.0      0.0   0.0      0.0  …  -0.0      -0.0      -0.0
  0.0      0.0   0.0      0.0   0.0      0.0     -0.0      -0.0      -0.0
  0.0      0.0   0.0      1.0   0.0      0.0     -0.0      -0.0      -0.0
  0.0      0.0   0.0      0.0   0.0      0.0     -0.0      -0.0      -0.0
  0.0      0.0   0.0      0.0   0.0      1.0     -0.0      -0.0      -0.0
  0.0      0.0   0.0      0.0   0.0      0.0  …  -0.0      -0.0      -0.0
  0.0      0.0   0.0      0.0   0.0      0.0     -0.0      -0.0      -0.0
  0.0      0.0   0.0      0.0   0.0      0.0     -1.73205  -0.0      -0.0
  0.0      0.0   0.0      0.0   0.0      0.0     -0.0      -0.0      -0.0
  0.0      0.0   0.0      0.0   0.0      0.0     -0.0      -1.0      -1.73205
  1.73205  0.0  -1.73205  0.0   0.0      0.0  …  -0.0      -0.0      -0.0
  1.0      0.0  -1.0      0.0   0.0      0.0     -0.0      -0.0      -0.0
 -1.41421  0.0   2.82843  0.0  -1.41421  0.0     -0.0      -0.0      -0.0
 -1.0      

In [157]:
Acl |> eigvals

10-element Vector{ComplexF64}:
   -2.0687760219908036 + 0.0im
    -1.467029518314265 - 0.24755381078639013im
    -1.467029518314265 + 0.24755381078639013im
   -1.1636174267858947 - 1.4687535674836465im
   -1.1636174267858947 + 1.4687535674836465im
   -1.1517937415128028 + 0.0im
   -0.9313053020371831 - 1.3069897795977856im
   -0.9313053020371831 + 1.3069897795977856im
 4.836423387876063e-16 - 1.0613698185394586e-8im
 4.836423387876063e-16 + 1.0613698185394586e-8im

In [158]:
Acl2 |> eigvals

20-element Vector{ComplexF64}:
     -3.067945801568088 + 0.0im
     -2.642641298987878 + 0.0im
     -2.181707673711644 - 1.5684931140552534im
     -2.181707673711644 + 1.5684931140552534im
    -1.9043019445276794 - 1.3513865115749395im
    -1.9043019445276794 + 1.3513865115749395im
    -1.0633956467028876 - 0.4763522564611544im
    -1.0633956467028876 + 0.4763522564611544im
    -0.8660254037846692 - 0.4999999999998823im
    -0.8660254037846692 + 0.4999999999998823im
    -0.4870502352357613 - 1.4331513121496193im
    -0.4870502352357613 + 1.4331513121496193im
   -0.45403330584804014 - 1.6403264456027662im
   -0.45403330584804014 + 1.6403264456027662im
    -0.4017566164041519 + 0.0im
    -0.3867876181052776 + 0.0im
   -0.09873863626693724 - 0.25621410120162363im
   -0.09873863626693724 + 0.25621410120162363im
 -2.6102245858457105e-8 + 0.0im
   2.610224633268355e-8 + 0.0im